# FLARE Query Engine

Adapted from the paper "Active Retrieval Augmented Generation"

Currently implements FLARE Instruct, which tells the LLM to generate retrieval instructions.

We create an adapted implementation:
- At the beginning, feed in user query and the existing answer.
- Generate a "lookahead" sentence that contains "search" tags. Parse this lookahead sentence and extract the search tags, and run those against a query engine to extract the answers. Insert into the lookahead template for an actual answer.


**NOTE**: this is a beta feature. The interface might change, and features may not be guaranteed to work. We're very open to feedback + suggestions!

In [6]:
import os
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from llama_index.query_engine import FLAREInstructQueryEngine
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    LLMPredictor,
    ServiceContext,
)

## Setup - Load in Data, Define Index, Configure ServiceContext

In [7]:
service_context = ServiceContext.from_defaults(
    llm_predictor=LLMPredictor(llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)),
    chunk_size=512
)

In [8]:
documents = SimpleDirectoryReader("../data/paul_graham").load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

## Define FLAREInstructQueryEngine

The FLAREInstructQueryEngine takes in an existing query engine as input.

In [9]:
index_query_engine = index.as_query_engine(similarity_top_k=2)

In [10]:
flare_query_engine = FLAREInstructQueryEngine(
    query_engine=index_query_engine,
    service_context=service_context,
    verbose=True
)

In [11]:
flare_query_engine

## Try running some queries! 

We showcase the sequential generation capabilities of the FLARE query engine.

We compare these queries against a naive vector-store-based query engine.

In [12]:
# flare query engine
response = flare_query_engine.query("Can you tell me about the author's trajectory in the startup world?")

Query: Can you tell me about the author's trajectory in the startup world?
Current response: 
Lookahead response: The author began his career in the startup world by [Search(What was the author's first startup venture?)]. After that, he went on to [Search(What other startups did the author work on?)]. He eventually became a successful entrepreneur, founding his own company [Search(What is the name of the author's company?)].
Updated lookahead response: The author began his career in the startup world by 
Current response:  The author began his career in the startup world by
Lookahead response: [Search(How did the author start his career in the startup world?)]. He then went on to found his own startup, which [Search(What was the author's startup and what did it do?)]. After the success of his startup, the author became a well-known figure in the startup community and began [Search(What other activities did the author engage in within the startup world?)].
Updated lookahead response:  w

In [13]:
print(response)

The author began his career in the startup world by writing essays and working on spam filters. He also hosted dinners for friends and bought a building in Cambridge to use as an office. He met Jessica Livingston at a party in 2003 and later discussed with her the need for changes in venture capital. Together, they founded Y Combinator, a startup accelerator that funds and mentors early-stage startups. After the success of Y Combinator, the author became a well-known figure in the startup community and began investing in startups and writing books on entrepreneurship. 


In [17]:
# (baseline) vector index query engine
response = index_query_engine.query("Can you tell me about the author's trajectory in the startup world?")

In [16]:
print(response)

The author has written essays on various topics, worked on spam filters, and bought a building in Cambridge to use as an office. They met Jessica Livingston at a party and later discussed the need for improvements in venture capital. The author gave a talk on starting a startup and realized they should stop procrastinating about angel investing. They then decided to start their own investment firm with Jessica and two other partners. They had no prior experience in angel investing but made what seemed like obvious choices and some of their actions turned out to be novel.


In [12]:
# flare query engine
response = flare_query_engine.query("Can you tell me about the author's journey in college and art school")

Query: Can you tell me about the author's journey in college and art school
Current response: 
Lookahead response: The author attended college and art school. During college, he took classes in [Search(What classes did the author take in college?)]. After college, he attended art school where he studied [Search(What did the author study in art school?)].
Updated lookahead response: The author attended college and art school. During college, he took classes in drawing, color, and design as part of the foundation program at RISD. After college, he attended art school where he studied 
Current response:  The author attended college and art school. During college, he took classes in drawing, color, and design as part of the foundation program at RISD. After college, he attended art school where he studied
Lookahead response: [Search(What did the author study in art school?)].
Updated lookahead response:  painting.
Current response: The author attended college and art school. During college

In [13]:
print(response)

The author attended college and art school. During college, he took classes in drawing, color, and design as part of the foundation program at RISD. After college, he attended art school where he studied painting. at RISD. or took fundamental classes like drawing, color, and design as part of the foundation program at RISD. He then went on to attend art school where he studied painting. 


In [18]:
# (baseline) vector index query engine
response = index_query_engine.query("Can you tell me about the author's journey in college and art school")

In [19]:
print(response)

Yes, the author applied to two art schools, RISD in the US and the Accademia di Belli Arti in Florence. They were accepted to RISD and had to do the foundation classes that summer. Later, they received a delayed invitation from the Accademia to take the entrance exam in Florence. The author decided to go to Florence and learn Italian, and eventually passed the entrance exam. They also took art classes at Harvard while still in a PhD program in computer science. The author struggled with wanting to be an artist but also being interested in computer science. They eventually found a way out of grad school by writing their dissertation in five weeks and reusing parts of their work on Lisp.
